Import libraries and set up API key

In [1]:
import json
import requests
import pandas as pd
from datetime import datetime, timedelta

# Insert your API key here
API_KEY = ''

Get user input

In [2]:
# Insert your API key here
API_KEY = str(input("Insert your API key here"))
#Ask user for input
num_data_points = int(input("How many data points do you want to load? "))
start_date = input("Enter the starting date (YYYY-MM-DD): ")

# Calculate end date based on start date and number of data points
start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
end_datetime = start_datetime + timedelta(days=num_data_points)
end_date = end_datetime.strftime("%Y-%m-%d")

Define the function to fetch data

In [ ]:
def fetch_glassnode_data(endpoint, asset='BTC'):
    url = f'https://api.glassnode.com/v1/metrics/{endpoint}'
    params = {
        'a': asset,
        'api_key': API_KEY,
        's': start_date,
        'u': end_date,
        'i': '24h'  # daily interval
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_json(response.text, convert_dates=['t'])
        column_name = endpoint.split('/')[-1]
        df = df.rename(columns={'t': 'timestamp', 'v': column_name})
        return df.set_index('timestamp')
    else:
        print(f"Error fetching data from {endpoint}: {response.status_code}")
        return pd.DataFrame()

Fetch price data

In [ ]:
# Fetch price data
price_df = fetch_glassnode_data('market/price_usd_close')
price_df.head()

Define endpoints and fetch feature data

In [ ]:
# List of endpoint URLs (add more as needed)
endpoints = [
    'indicators/sopr',
    'market/marketcap_usd',
    'transactions/transfers_volume_sum'
]

# Fetch and combine feature data
features_df = pd.DataFrame()
for endpoint in endpoints:
    df = fetch_glassnode_data(endpoint)
    if features_df.empty:
        features_df = df
    else:
        features_df = features_df.join(df, how='outer')

features_df.head()

Combine data and handle missing values

In [ ]:
# Concatenate price and features dataframes
combined_df = pd.concat([price_df, features_df], axis=1)

# Handle missing values
combined_df = combined_df.dropna()

# Display the result
combined_df.head()

 Additional analysis or visualization (optional)

In [ ]:
# You can add more cells here for further analysis or visualization
combined_df.describe()